In [8]:
from dotenv import load_dotenv, find_dotenv
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage

from langchain_community.tools.tavily_search import TavilySearchResults
import os
import getpass
from langchain_core.prompts import ChatPromptTemplate
import sys

In [2]:
# set the envionment variables
_ = load_dotenv(find_dotenv())
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )

os.environ["LANGSMITH_PROJECT"] = "default"
# if "LANGSMITH_PROJECT" not in os.environ:
#     os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
#         prompt='Enter your LangSmith Project Name (default = "default"): '
#     )
#     if not os.environ.get("LANGSMITH_PROJECT"):
#         os.environ["LANGSMITH_PROJECT"] = "default"

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(
        prompt="Enter your Google API key (required if using Google Gemini): "
    )


In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
#model = init_chat_model("gpt-4o-mini", model_provider="openai")



In [5]:
model.invoke([HumanMessage(content="Hi! I'm Bob")])



AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6d8e752b-d24f-4ed7-8041-e8acafb09607-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

In [ ]:
# does not have any memory
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As a large language model, I don't have access to personal information. Therefore, I don't know your name. You haven't told me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e0fa6742-805c-4388-b319-3ac05d7f7ba0-0', usage_metadata={'input_tokens': 6, 'output_tokens': 34, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}})

In [9]:

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--65b5894c-73a9-4f46-a182-7154f2d8d51d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 12, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [11]:
config = {"configurable": {"thread_id": "abc123"}}



In [12]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state



================================== Ai Message ==================================

Hi Bob! It's nice to meet you. How can I help you today?


In [13]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. You just told me!


change the thread to start a fresh conversation

In [14]:
config = {"configurable": {"thread_id": "abc234"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a large language model, I don't have access to personal information about you, including your name. You haven't told me your name.


However, we can always go back to the original conversation (since we are persisting it in a databases


In [15]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Bob. So, your name is Bob.
